<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1MvzejZCb_26jS-EqqfiWEpYQWq8xoKKc?usp=sharing)

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions
- No coding experience required
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)




# Personalized AI Search with Mem0, LangChain, and Tavily

## Getting Started

This notebook demonstrates how to build a personalized search assistant that combines:
- **Mem0**: For storing and retrieving user context and preferences
- **Tavily**: For performing web searches optimized for LLMs
- **LangChain**: For orchestrating the AI agent workflow

The system works by:
1. Storing user context and preferences in Mem0
2. Using that context to personalize search queries
3. Retrieving relevant information from the web via Tavily
4. Using an AI agent to synthesize personalized responses

Let's get started by installing the required dependencies.

In [1]:
# Install required packages
!pip install mem0ai langchain_tavily langchain_openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.3/122.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 12.7 MB/s eta 0:00:00


## Setting up API Keys

We need to set up API keys for the services we'll be using:
- **OpenAI API Key**: For the language model (gpt-4o-mini)
- **Mem0 API Key**: For memory management
- **Tavily API Key**: For web search capabilities

In Google Colab, we use `userdata.get()` to securely retrieve these keys from the Secrets manager.

In [2]:
# Set up API keys
import os
from google.colab import userdata

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Set Mem0 API key
os.environ['MEM0_API_KEY'] = userdata.get('MEM0_API_KEY')

# Set Tavily API key
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

## Initializing Components

Here we initialize the core components of our system:
- **ChatOpenAI**: Our language model (gpt-4o-mini with a temperature of 0.2 for balanced creativity)
- **MemoryClient**: The Mem0 client for managing user memories
- **TavilySearch**: Our search tool with advanced search depth and up to 10 results

In [3]:
from mem0 import MemoryClient
from langchain_tavily import TavilySearch
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

# Initialize components
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
mem0_client = MemoryClient()

# Tavily search tool
tavily_search = TavilySearch(max_results=10, search_depth="advanced")

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## Adding User Memory

We'll add some initial context about our user. In this case, we're storing that the user lives in Greater Noida, Delhi. This information will be used to personalize future search results.

The `add()` method stores memories associated with a specific user ID, and `list()` retrieves all stored memories for that user.

In [7]:
# Add user history for personalization
client = MemoryClient()

client.add(
    [
        {"role": "user", "content": "I live in Greater Noida, Delhi."}
    ],
    user_id="john"
)

# View added memories
filters = {
   "AND": [
      {
         "user_id": "john"
      }
   ]
}

all_memories = client.get_all(version="v2", filters=filters, page=1, page_size=50)

/usr/local/lib/python3.12/dist-packages/mem0/client/utils.py:21: DeprecationWarning: output_format='v1.0' is deprecated therefore setting it to 'v1.1' by default. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


In [8]:
all_memories

{'count': 1,
 'next': None,
 'previous': None,
 'results': [{'id': 'f2d1a125-7e6c-4034-a04a-3bb55b8719ed',
   'memory': 'User Lives in Greater Noida, Delhi',
   'user_id': 'john',
   'metadata': None,
   'categories': ['personal_details'],
   'created_at': '2025-08-28T02:36:55.967625-07:00',
   'updated_at': '2025-08-28T02:36:56.030710-07:00',
   'expiration_date': None,
   'structured_attributes': {'day': 28,
    'hour': 9,
    'year': 2025,
    'month': 8,
    'minute': 36,
    'quarter': 3,
    'is_weekend': False,
    'day_of_week': 'thursday',
    'day_of_year': 240,
    'week_of_year': 35}}]}

## Retrieving Context for Queries

When a user makes a query, we use Mem0's `search()` method to find relevant context from their stored memories. This ensures that search results are personalized based on what we know about the user.

In this example, when the user asks about "best coffee shops nearby", we retrieve their location context to provide location-relevant results.

In [9]:
# New user search query
query = "best coffee shops nearby"

# Retrieve user context relevant to this query
user_context = "\n".join(
    [f"- {m['memory']}" for m in mem0_client.search(query=query, user_id="john")]
)

print("Query:", query)
print("\nRelevant user context:")
print(user_context if user_context else "No relevant context found")

Query: best coffee shops nearby

Relevant user context:
- User Lives in Greater Noida, Delhi


## Creating the AI Agent

We create an AI agent using LangChain that will:
1. Use our prompt template which includes the user context
2. Have access to the Tavily search tool
3. Be powered by our OpenAI language model

The agent will use this setup to intelligently search the web and synthesize personalized responses based on both the user's query and their stored context.

In [10]:
# Create prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a personalized search assistant.
USER CONTEXT: {user_context}
Tailor results to preferences like dietary needs, family context, and lifestyle."""),
    MessagesPlaceholder("messages"),
    MessagesPlaceholder("agent_scratchpad")
])

# Create agent
agent = create_openai_tools_agent(
    llm=llm,
    tools=[tavily_search],
    prompt=prompt
)

# Create executor
executor = AgentExecutor(
    agent=agent,
    tools=[tavily_search],
    return_intermediate_steps=True
)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## Running the Agent

Now we execute the agent with our query and the retrieved user context. The agent will:
1. Use the Tavily tool to search for relevant information
2. Process the search results
3. Generate a personalized response based on both the search results and user context

The `return_intermediate_steps=True` parameter allows us to see the search results in the next step.

In [11]:
# Run the agent with the query
response = executor.invoke(
    {
        "messages": [HumanMessage(content=query)],
        "user_context": user_context
    }
)

print("Agent response:")
print(response["output"])

Agent response:
Here are some of the best coffee shops in Greater Noida:

1. **[Wanderlog - Best Coffee Shops and Cafes](https://wanderlog.com/list/geoCategory/282945/best-coffee-shops-and-best-cafes-in-greater-noida)**: This list includes popular spots like:
   - Alma Bakery & Cafe
   - Starbucks
   - The Desi Firangi
   - Cold Rock Cafe
   - Brewtiful
   - The Little Pink Café

2. **[Tripadvisor - The 10 Best Cafés](https://www.tripadvisor.com/Restaurants-g2140594-c8-Greater_Noida_Gautam_Buddha_Nagar_District_Uttar_Pradesh.html)**: Highlights include:
   - Cafe Honey Hut
   - Bakers N Brewers
   - The Little Pink Café

3. **[Zomato - Cafe Restaurants in Greater Noida](https://www.zomato.com/ncr/greater-noida-restaurants/cafes)**: Features various cafes like:
   - Starbucks Coffee
   - D' Wood Cafe
   - Modern Basket Cafe

4. **[Party Witty - Top 7 Cafes](https://partywitty.com/blogs/top-7-cafes-in-greater-noida-for-weekend-brunch-&-bachelorette-scenes)**: A curated list of cafes perf

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## Processing Search Results

We can examine the raw search results from Tavily that the agent used to generate its response. This shows the actual sources and information that informed the personalized answer.

In [12]:
# Extract the tavily results
tavily_results = response["intermediate_steps"][0][1]["results"]

print(f"Found {len(tavily_results)} results:\n")

# Print each result's title and content
for i, r in enumerate(tavily_results, 1):
    print(f"{i}. {r['title']}")
    print(f"   URL: {r['url']}")
    print(f"   Snippet: {r['content']}\n")

Found 10 results:

1. The 35 best coffee shops and best cafes in Greater Noida - Wanderlog
   URL: https://wanderlog.com/list/geoCategory/282945/best-coffee-shops-and-best-cafes-in-greater-noida
   Snippet: ## 22Alma Bakery & Cafe Noida sector 104

## 23Starbucks

## 24The Desi Firangi

## 25Cold Rock Cafe

## 26Chelvies Coffee | Noida 104

## 27Imperfecto Rainbow

## 28Cafe Delhi Heights

## 29Saltoro Coffee Roasters

## 30Tan Coffee

## 31Pind balluchi

## 32Brewtiful

## 33The Little Pink Café - Coffee, Cakes and Small-Bites

## 34Bakers N Brewers

## 35Mera Pizza

Popular

Food

Popular [...] ## 6Bark Street | Dog Food & Grooming Cafe in Noida

## 7Uncle's Patty

## 8Sparrows At Home Cafe

## 9Spezia Bistro

## 10Roastery Coffee House | Noida

## 11Café Ciro

## 12Scarlet Bar

## 13Palm Garden Restaurant (The Circle Cafe)

## 14Creme Castle

## 15Cafe Street 1

## 16Sip & Social

## 17Blue Tokai Coffee Roasters | Noida Sector 104

## 18Chaayos Cafe at Ithum Galleria

## 19The Reade

## Complex Use Case: Personalized Travel Planning Assistant

Let's build a more sophisticated example that demonstrates the full power of our personalized search system. We'll create a travel planning assistant that:

1. Stores detailed user preferences (dietary restrictions, interests, budget)
2. Uses those preferences to find highly personalized travel recommendations
3. Synthesizes information from multiple sources into a cohesive travel plan

This example shows how the system can handle complex, multi-faceted queries that require combining various pieces of user context.

In [16]:
# Create a new user with detailed travel preferences
traveler_context = [
    {"role": "user", "content": "I'm planning a 5-day trip to Kyoto, Japan in November"},
    {"role": "user", "content": "I'm a vegetarian and prefer traditional Japanese cuisine"},
    {"role": "user", "content": "I'm interested in visiting temples, gardens, and experiencing traditional tea ceremonies"},
    {"role": "user", "content": "I prefer staying in ryokans (traditional inns) over hotels"},
    {"role": "user", "content": "My budget is $200 per day excluding accommodation"},
    {"role": "user", "content": "I enjoy walking and prefer neighborhoods with narrow streets and traditional architecture"}
]

# Add context to Mem0 for our new user
mem0_client.add(traveler_context, user_id="traveler_alex")

# View the stored context
filters = {
   "AND": [
      {
         "user_id": "traveler_alex"
      }
   ]
}

all_memories = client.get_all(version="v2", filters=filters, page=1, page_size=50)

/usr/local/lib/python3.12/dist-packages/mem0/client/utils.py:21: DeprecationWarning: output_format='v1.0' is deprecated therefore setting it to 'v1.1' by default. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


In [17]:
all_memories

{'count': 7,
 'next': None,
 'previous': None,
 'results': [{'id': '8f7a7f1c-bbea-4f97-923a-5ca1bc392c98',
   'memory': 'User enjoys walking and prefers neighborhoods with narrow streets and traditional architecture',
   'user_id': 'traveler_alex',
   'metadata': None,
   'categories': None,
   'created_at': '2025-08-28T03:23:47.576503-07:00',
   'updated_at': '2025-08-28T03:23:47.639696-07:00',
   'expiration_date': None,
   'structured_attributes': {'day': 28,
    'hour': 10,
    'year': 2025,
    'month': 8,
    'minute': 23,
    'quarter': 3,
    'is_weekend': False,
    'day_of_week': 'thursday',
    'day_of_year': 240,
    'week_of_year': 35}},
  {'id': 'f1511a6e-6ea7-4a40-b4a6-e5e7e7d17723',
   'memory': "User's budget is $200 per day excluding accommodation",
   'user_id': 'traveler_alex',
   'metadata': None,
   'categories': None,
   'created_at': '2025-08-28T03:23:46.333861-07:00',
   'updated_at': '2025-08-28T03:23:46.396306-07:00',
   'expiration_date': None,
   'structure

In [18]:
# Define a complex travel query that requires combining multiple preferences
complex_travel_query = """
Based on my preferences, please provide:
1. A list of 3 highly-rated ryokans in Kyoto with vegetarian-friendly options
2. 5 must-visit temples and gardens that are within walking distance of each other
3. Recommendations for authentic vegetarian Japanese restaurants near these locations
4. Information about tea ceremony experiences that align with my interest in tradition
5. Any seasonal events or festivals happening in November that would match my interests
"""

# Retrieve relevant context from Mem0
travel_context = "\n".join(
    [f"- {m['memory']}" for m in mem0_client.search(query=complex_travel_query, user_id="traveler_alex")]
)

print("Complex travel query:")
print(complex_travel_query)
print("\nRelevant user context:")
print(travel_context)

# Run the agent with our complex travel query
print("\nGenerating personalized travel recommendations...")
complex_response = executor.invoke(
    {
        "messages": [HumanMessage(content=complex_travel_query)],
        "user_context": travel_context
    }
)

print("\nPersonalized travel recommendations:")
print(complex_response["output"])

Complex travel query:

Based on my preferences, please provide:
1. A list of 3 highly-rated ryokans in Kyoto with vegetarian-friendly options
2. 5 must-visit temples and gardens that are within walking distance of each other
3. Recommendations for authentic vegetarian Japanese restaurants near these locations
4. Information about tea ceremony experiences that align with my interest in tradition
5. Any seasonal events or festivals happening in November that would match my interests


Relevant user context:
- User is planning a 5-day trip to Kyoto, Japan in November
- User is interested in visiting temples, gardens, and experiencing traditional tea ceremonies
- User prefers staying in ryokans (traditional inns) over hotels
- User prefers traditional Japanese cuisine
- User enjoys walking and prefers neighborhoods with narrow streets and traditional architecture
- User is a vegetarian

Generating personalized travel recommendations...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)



Personalized travel recommendations:
Here’s a tailored itinerary for your 5-day trip to Kyoto in November, focusing on your interests in temples, gardens, traditional tea ceremonies, vegetarian cuisine, and seasonal events.

### 1. Highly-Rated Ryokans in Kyoto with Vegetarian-Friendly Options
- **Rangetsu**: Located near the Arashiyama bamboo grove, this ryokan offers private onsens and traditional Japanese meals, including vegetarian options. [More Info](https://www.erikastravelventures.com/japanese-ryokan-with-vegetarian-options/)
- **Kyoto Nanzenji Ryokan Yachiyo**: Offers vegan and vegetarian breakfast and dinner options. Prices start around $627 per night for two people. [More Info](https://www.erikastravelventures.com/japanese-ryokan-with-vegetarian-options/)
- **Kyonoyado Gekkoan**: Features luxury suites with kaiseki meals that include vegan and vegetarian options. Prices start at $583 per night for two people. [More Info](https://www.erikastravelventures.com/japanese-ryokan-

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [19]:
# Examine the raw search results that informed our travel recommendations
travel_results = complex_response["intermediate_steps"][0][1]["results"]

print(f"Found {len(travel_results)} sources for travel recommendations:\n")

# Print each result's title and content
for i, r in enumerate(travel_results, 1):
    print(f"{i}. {r['title']}")
    print(f"   URL: {r['url']}")
    print(f"   Relevance Score: {r['score']:.2f}")
    print(f"   Snippet: {r['content']}\n")

Found 10 sources for travel recommendations:

1. 21 Japanese Ryokan with Vegetarian Kaiseki Options
   URL: https://www.erikastravelventures.com/japanese-ryokan-with-vegetarian-options/
   Relevance Score: 0.92
   Snippet: Rangetsu is a highly-rated Kyoto ryokan with private onsen located in the western part of the city, near Arashiyama bamboo grove. This traditional ryokan has rooms with Japanese futon bedding on tatami mats, as well as traditional Japanese seating areas on _zabuton_ cushions. The private onsen attached to some rooms are located on an outdoor terrace, and is fed from hot springs water from the Arashiyama hot springs. The private onsen overlooks the nearby Katsura River and the bamboo forest, and [...] This Kyoto ryokan with vegetarian meal options is about as traditional as it gets, and is perfect for visitors seeking an authentic Japanese experience in Kyoto!

Check the latest rates and availabilityhere.

21. Kyoto Nanzenji Ryokan Yachiyo - Vegan and Vegetarian Optio

## Conclusion

This notebook demonstrated how to build a powerful personalized search assistant by combining:

1. **Mem0** for persistent user context storage and retrieval
2. **Tavily** for high-quality, LLM-optimized web search
3. **LangChain** for orchestrating the AI agent workflow

Key takeaways:
- User context significantly improves the relevance of search results
- The system can handle both simple queries and complex, multi-faceted requests
- Personalization works best when detailed user preferences are stored
- The agent can synthesize information from multiple sources into coherent responses

Possible extensions:
- Adding more memory types (preferences, past interactions, feedback)
- Integrating more tools (maps, booking APIs, weather services)
- Implementing feedback loops to improve personalization over time
- Adding support for multiple users with different preference profiles